In [ ]:
'''
My tensorflow version:  2.5.0
Is tensorflow-gpu available:  False
My sklearn version:  0.24.2
My numpy version:  1.19.5
My pandas version:  1.2.0
'''
from mylibs.training_part import show_lib_version
show_lib_version()

### Retrive data for feature engineering

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
ts_code = '000858.SZ'

In [ ]:
df = pd.read_csv(f'./data/{ts_code}/basic_data.csv', index_col='trade_date', parse_dates=True)
df['label'] = np.where(df['close'].shift(-1) > df['close'] * 1.00025, 1, 0)
'''the last day has no label'''
df = df[:-1]

### Heat map

In [ ]:
from mylibs.feature_engineering_part import heat_map

In [ ]:
heat_map(df)

### K-means

In [ ]:
import seaborn as sn
from mylibs.feature_engineering_part import plot_inertia, get_clusters, random_choose_features, pca_k

In [ ]:
'''use elbow method to decide how many features to choose using k means'''
plot_inertia(df.drop('label', axis=1))

In [ ]:
'''decide to use 20 clusters'''
df_clusters = get_clusters(df.drop('label', axis=1), 20)

### Random choice

In [ ]:
'''random choose one feature from each cluster'''
prediction_data_k_means_random_feature = random_choose_features(df, df_clusters)
prediction_data_k_means_random_feature.dropna(axis=0, inplace=True)
prediction_data_k_means_random_feature.to_csv(f'./data/{ts_code}/prediction_data_k_means_random_feature.csv', index=True, sep=',')

In [ ]:
heat_map(prediction_data_k_means_random_feature)

In [ ]:
prediction_data_k_means_random_feature = pd.merge(prediction_data_k_means_random_feature, df[['label']], how='left', 
                                                  left_index=True, right_index=True)
sn.pairplot(prediction_data_k_means_random_feature, hue='label', palette='bright');

### do PCA in each cluster

In [ ]:
'''use PCA method to decrease the dimension to one in each cluster'''
prediction_data_k_pca = pca_k(df, df_clusters)
prediction_data_k_pca.to_csv(f'./data/{ts_code}/prediction_data_k_pca.csv', index=True, sep=',')

In [ ]:
prediction_data_k_pca = pd.DataFrame(prediction_data_k_pca.values, columns=[str(i) for i in range(len(prediction_data_k_pca.columns))],
                                    index= prediction_data_k_pca.index)
prediction_data_k_pca = pd.merge(prediction_data_k_pca, df[['label']], how='left', left_index=True, right_index=True)

In [ ]:
heat_map(prediction_data_k_pca)

In [ ]:
sn.pairplot(prediction_data_k_pca, hue='label', palette='bright');